# PROGETTO FINALE

## FIFA Talent Scouting

### Importo le librerie necessarie

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn import preprocessing
from sklearn.model_selection import learning_curve, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Carico il file nel DataFrame


In [ ]:
giocatori = pd.read_csv("players_20.csv" )

### Visiono il DataFrame

In [ ]:
#opzione che mi permette di mostrare tutte le colonne del csv
pd.set_option('display.max_columns', None)   #None mi fa vedere tutte le colonne, se metto un numero N mi
#fa vedere N colonne

#stampo la parte iniziale del file per poterne visionare le caratteristiche
print(giocatori.shape)
giocatori.head()

### Inizio processo di EDA

In [ ]:
#Andiamo a considerare solo le features più importanti per lo scouting

giocatori=giocatori[["age","overall","potential","value_eur","wage_eur","weak_foot","skill_moves"]]

#Voglio tenere solo i giocatori al di sotto di una certa età e un certo overall per effettuare lo scouting

giocatori = giocatori.drop(giocatori[giocatori.age>25].index)
giocatori = giocatori.drop(giocatori[giocatori.overall>75].index)
#giocatori = giocatori.drop(giocatori[giocatori.age<18].index)
#giocatori = giocatori.drop(giocatori[giocatori.overall<52].index)

print(giocatori.shape)
giocatori.head()



In [ ]:
#Vediamo se abbiamo valori nulli o non definiti
giocatori.isna().sum()

In [ ]:
print(giocatori.groupby(["age"]).size())
print()
giocatori.groupby(["overall"]).size()

In [ ]:
#Andiamo a vedere quali sono le features che influenzano di più la nostra classe, che ricordiamo essere "Potential"

correlation=giocatori.corr()

a4_dims = (10, 7)
fig, ax = plt.subplots(figsize=a4_dims)

sns.heatmap(data=correlation, square=True, cmap='YlOrRd', ax=ax, annot=True)

In [ ]:
#x = giocatori.drop('potential',1)
#y = giocatori.potential

#x.shape, y.shape

x=np.array(giocatori.drop(['potential'],1))
y=np.array(giocatori['potential'])

#Senza lo Scaler l'algoritmo SGD non funziona
scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.3, random_state=56)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape

### Creo il modello per la Linear Regression

In [ ]:
#Richiamo l'algoritmo LinearRegression e creo il modello sul dataset di train
linreg=LinearRegression()
linreg.fit(x_train, y_train)

#Valutiamo l'accuratezza del modello di regressione sul dataset di train e di test
accuracy_train=linreg.score(x_train, y_train)
accuracy_test=linreg.score(x_test,y_test)  
print("L'accuratezza per il dataset di train con splitting è:", accuracy_train)
print("L'accuratezza per il dataset di test con splitting è:", accuracy_test)

#Andiamo a vedere l'accuratezza inserendo una cross-validation al posto dello splitting, per evitare l'overfitting
#score=cross_val_score(linreg, x, y, cv=10, scoring="r2")
#print("L'accuratezza per il dataset intero con cross-validation è", score.mean())

mse_train=mean_squared_error(y_train, linreg.predict(x_train))   
mse_test=mean_squared_error(y_test, linreg.predict(x_test))
print("The mean squared error (MSE) on train set: {:.4f}".format(mse_train))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse_test))

#score_MSE=cross_val_score(linreg, x, y, cv=10, scoring="neg_mean_squared_error")
#print("Il MSE per il dataset intero con cross-validation è", -score_MSE.mean())


### Creo il modello per la Rete Neurale

In [ ]:
#Creo il modello con l'algoritmo rete neurale

MLP=MLPRegressor(tol=0.00005, max_iter=1000, hidden_layer_sizes=10)
MLP.fit(x_train, y_train)

#Valutiamo l'accuratezza del modello di regressione sul dataset di train e di test
accuracy_train=MLP.score(x_train, y_train)
accuracy_test=MLP.score(x_test,y_test)  
print("L'accuratezza per il dataset di train con splitting è:", accuracy_train)
print("L'accuratezza per il dataset di test con splitting è:", accuracy_test)

#Andiamo a vedere l'accuratezza inserendo una cross-validation al posto dello splitting, per evitare l'overfitting
#score=cross_val_score(MLP, x, y, cv=10, scoring="r2")
#print("L'accuratezza per il dataset intero con cross-validation è", score.mean())

mse_train=mean_squared_error(y_train, MLP.predict(x_train))   
mse_test=mean_squared_error(y_test, MLP.predict(x_test))
print("The mean squared error (MSE) on train set: {:.4f}".format(mse_train))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse_test))


#score_MSE=cross_val_score(MLP, x, y, cv=10, scoring="neg_mean_squared_error")
#print("Il MSE per il dataset intero con cross-validation è", -score_MSE.mean())


In [ ]:
MLP.predict(x_test)

In [ ]:
#Realizziamo le learning curves

_sizes=[i for i in range(1, 4600,10)]
train_sizes=np.array(_sizes)  # Relative sizes
scoring='neg_mean_squared_error'

lr=LinearRegression()

train_sizes_abs, train_scores, validation_scores = learning_curve(
    lr, x_train, y_train, train_sizes=train_sizes, cv=10, scoring=scoring, shuffle=True)


train_scores_mean = -train_scores.mean(axis = 1)
validation_scores_mean = -validation_scores.mean(axis = 1)

In [ ]:
#Plottiamo le learning curves

plt.style.use('seaborn')
plt.plot(train_sizes_abs, train_scores_mean, label = 'Training error')
plt.plot(train_sizes_abs, validation_scores_mean, label = 'Cross-Validation error')
plt.ylabel('MSE', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a linear regression model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,10)
plt.xlim(0,400)

### Creo il modello per il Random Forest Classifier 

In [ ]:
giocatori.head(50)

In [ ]:
giocatori["potential_dis"]=""
numeroRighe= giocatori.shape[0] 
for i in range(0,numeroRighe):
    if giocatori.iat[i,2]-giocatori.iat[i,1] > 10:
        giocatori.iat[i,7]="Ottima crescita >10"
    elif giocatori.iat[i,2]-giocatori.iat[i,1] > 7 and giocatori.iat[i,2]-giocatori.iat[i,1] <= 10:
        giocatori.iat[i,7]="Buona crescita ]7,10]"
    elif giocatori.iat[i,2]-giocatori.iat[i,1] > 3 and giocatori.iat[i,2]-giocatori.iat[i,1] <= 7:
        giocatori.iat[i,7]="Normale crescita ]3,7]" 
    elif giocatori.iat[i,2]-giocatori.iat[i,1] <= 3:
        giocatori.iat[i,7]="Pessima crescita <=3" 

In [ ]:
giocatori = giocatori[[ "age","overall","value_eur","wage_eur","weak_foot","skill_moves","potential_dis"]]
giocatori.head(5)

In [ ]:
X = giocatori.drop('potential_dis',axis=1)
y = giocatori['potential_dis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [ ]:
regressor = RandomForestClassifier(n_estimators=20, random_state=1)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

### Salvataggio Modelli 

In [ ]:
#Salviamo i modelli di machine learning

#Modello regressione lineare
filename = 'FifaModelLinReg.sav'    
pickle.dump(linreg, open(filename, 'wb'))

#Modello rete neurale
filename = 'FifaModelMLP.sav'    
pickle.dump(MLP, open(filename, 'wb'))

#Modello random forest classifier
filename = 'FifaModelRanForClas.sav'    
pickle.dump(regressor, open(filename, 'wb'))



################################## CODICE PER CARICARE I MODELLI #########################################

# Carico il modello di regressione lineare precedentemente salvato
#loaded_model = pickle.load(open('FifaModelLinReg.sav', 'rb'))
#result = loaded_model.score(x, y)
#print(result)

#Carico il modello di rete neurale precedentemente salvato
#loaded_model = pickle.load(open('FifaModelMLP.sav', 'rb'))
#result = loaded_model.score(x, y)
#print(result)

#Carico il modello di rete neurale precedentemente salvato
#loaded_model = pickle.load(open('FifaModelRanForClas.sav', 'rb'))
#result = loaded_model.score(x, y)
#print(result)
